In [1]:
import ee
ee.Authenticate()
ee.Initialize()
print(ee.Image("NASA/NASADEM_HGT/001").get("title").getInfo())

Enter verification code: 4/1AbUR2VMs_9pxJZs3f-O30r2inCB_T07jrvGExO50cxQppZVoEcmjygqzTG4

Successfully saved authorization token.
NASADEM: NASA NASADEM Digital Elevation 30m


In [3]:
import datetime
ee_date = ee.Date('2020-01-01')
py_date = datetime.datetime.utcfromtimestamp(ee_date.getInfo()['value']/1000.0)
py_date = datetime.datetime.utcnow()
ee_date = ee.Date(py_date)


In [6]:
my_image = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_034033_20000913')

task = ee.batch.Export.image.toDrive(image=my_image,  # an ee.Image object.
                                     region=my_geometry,  # an ee.Geometry object.
                                     description='mock_export',
                                     folder='gdrive_folder',
                                     fileNamePrefix='mock_export',
                                     scale=1000,
                                     crs='EPSG:4326')


NameError: name 'my_geometry' is not defined

# Example from Satellite Imagery using Google Earth Engine and Python

https://www.askpython.com/python/examples/satellite-imagery-google-earth

In [17]:
collection = ee.ImageCollection('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS')

# 4. Working with the ImageCollection

The most important aspect that separates normal photos from satellite images is the spectrum of the images is way beyond the normal 0-255 range. The images have different bands which can be compared to the channels(RGB) of the normal image. Each band captures a different electromagnetic spectrum.

In [18]:
first_image = collection.first()

In [19]:
# Now we see what bands each image in the image collection have:
first_image.bandNames().getInfo()

['avg_vis', 'stable_lights', 'cf_cvg', 'avg_lights_x_pct']

We need the stable_lights band for visualization. Selecting one band from each image from the collection is time consuming, but luckily earth-engine offers a way to select one band from a collection of images.

In [5]:
collection = collection.select('stable_lights')

Now each image in the image collection has a single band. To visualize the change over time will be needing another band that will represent the time.

In [6]:
first_image = collection.first()
 
# Get number of years after 1991 from the first image
year = ee.Date(first_image.get('system:time_start')).get('year').subtract(1991)
 
# Turn it into a band
img1 = ee.Image(year).byte().addBands(first_image)
 
# A new band appears in the new image
img1.bandNames().getInfo()

['constant', 'stable_lights']

We need to do this for every image in the collection. So we create a function and map it over the collection.

In [7]:
def createTimeBand(img):
    year = ee.Date(img.get('system:time_start')).get('year').subtract(1991)
    return ee.Image(year).byte().addBands(img)
 
# Map it to the collection to create a new collection
collection = collection.map(createTimeBand)

# 5. Visualize the night-lights

Now that we have our ImageCollection ready to be displayed, we need a UI to display the images. There are a lot of UI options available like Folium, matplotlib, PIL, etc.

In this tutorial, we have chosen Folium. Folium is a python library based on leaflet.js (open-source JavaScript library for mobile-friendly interactive maps) that you can use to make interactive maps. Folium supports WMS, GeoJSON layers, vector layers, and tile layers which make it very convenient and straightforward to visualize the data we manipulate with python.

Now we need to define how to display the google earth images in folium. Here is a function that comes very handy to use with Folium and Earth-Engine.

In [9]:
# Define a method for displaying Earth Engine image tiles to folium map.
import folium
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data © <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)
 
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

Now we can create a map, giving the initial coordinates and the zoom factor. Don’t worry if you get it a little wrong. Folium is interactive, which means you can change the location and zoom while you are visualizing the map.

In [10]:
night_map = folium.Map(location=[37.5010, -102.1899], zoom_start=4.5)

We need to set some parameters for displaying the image, such as the bands and colour preferences of each. Here is a default setting that you might like.

In [11]:
params = {
    'bands' :  ['scale', 'offset', 'scale'],
    'min' : 0,
    'max' : [0.18, 20, -0.18],
}

Before we visualize, we have one final step. We need to reduce the image using LinearFit method which computes the least squares estimate of a linear function of one variable with a constant term.

In [12]:
night_light = collection.reduce(ee.Reducer.linearFit())

Add the image and parameters to the map and display the map.

In [14]:
# Add the image layer to the night_map
night_map.add_ee_layer(night_light, params, 'test')
 
# Display the map
display(night_map)